In [3]:
import os
from tqdm import  tqdm

all_conc = os.listdir("./data/raw_data/CONC/")
all_conc = [item for item in all_conc if not item.startswith('.')]

In [ ]:
all_dataset = {}

for conc in tqdm(all_conc):
    conc_name = conc.split(".")[0]
    conc_df = pd.read_csv(os.path.join("./data/raw_data/CONC/", conc))

    chemical_columns = [col for col in conc_df.columns if re.match(r'CONC\(\d+\)', col)]
    concentration_pairs = {}

    for i in range(len(conc_df) - 1):
        current_row = conc_df.iloc[i]
        next_row = conc_df.iloc[i + 1]
        
        current_jdate, current_jtime = current_row['JDATE'], current_row['JTIME']
        # if current_jdata is 2019001, skip
        if int(current_jdate) == 2019001: continue
        next_jdate, next_jtime = next_row['JDATE'], next_row['JTIME']
        
        if current_jdate == next_jdate and next_jtime == current_jtime:
            initial_conc = current_row[chemical_columns].values
            final_conc = next_row[chemical_columns].values
            concentration_pairs[(int(current_jdate), int(current_jtime))] = {'initial': initial_conc, 'final': final_conc}
    df = pd.DataFrame(concentration_pairs)
    
    all_dataset[conc_name] = concentration_pairs

In [5]:
import pickle as pkl
with open("./data/processed_data/all_dataset.pkl", "wb") as f:
    pkl.dump(all_dataset, f)